In [ ]:
# ! pip install python_hiccup

In [ ]:
# ! pip install edn_format

In [ ]:
# ! pip install hissp

In [ ]:
from functools import partial
import edn_format

def walk(inner, outer, coll):
    if isinstance(coll, edn_format.immutable_list.ImmutableList):
        return outer([inner(e) for e in coll])
    elif isinstance(coll, edn_format.immutable_dict.ImmutableDict):
        return outer(dict([inner(e) for e in iter(coll.items())]))
    elif isinstance(coll, tuple):
        return outer(tuple([inner(e) for e in coll]))
    else:
        return outer(coll)

def postwalk(fn, coll):
    return walk(partial(postwalk, fn), fn, coll)

In [ ]:
def simpleExecute(coll):
    if isinstance(coll, edn_format.edn_lex.Symbol):
        return globals().get(str(coll))
    elif isinstance(coll, edn_format.edn_lex.Keyword):
        return str(coll)[1:]
    elif isinstance(coll, tuple):
        f = coll[0]
        if isinstance(f, str):
            return coll
        else:
            return f(*coll[1:])
    else:
        return coll

def tT(*d):
    return d
    
def ednToHissp(edn):
    if isinstance(edn, edn_format.edn_lex.Symbol):
        return str(edn)    
    elif isinstance(edn, edn_format.edn_lex.Keyword):
        return ("quote", str(edn)[1:])
    elif isinstance(edn, str):
        return ("quote",edn)
    elif isinstance(edn, list):
        return tuple(["tT"] + edn)
    elif isinstance(edn, dict):
        return ("dict", tuple(["tT"] + [("tT",) + e for e in iter(edn.items())]))
    else:
        return edn

In [ ]:
from IPython.core.magic import (Magics, magics_class, cell_magic)
from IPython.display import display, HTML
from python_hiccup.html import render
from hissp import readerless
from hissp import _macro_

def html(markupString):
    display(HTML(render(markupString)))
    return None
    
@magics_class
class MyMagics(Magics):
    @cell_magic
    def inter(self, line, cell):
        return postwalk(simpleExecute, edn_format.loads(cell))
        
    @cell_magic
    def hissp(self, line, cell):
        erg=eval(readerless(postwalk(ednToHissp, edn_format.loads(cell))))
        return erg

    @cell_magic
    def hissp0(self, line, cell):
        erg=postwalk(ednToHissp, edn_format.loads(cell))
        return erg
        
    @cell_magic
    def hissp1(self, line, cell):
        erg=readerless(postwalk(ednToHissp, edn_format.loads(cell)))
        return print(erg)

In [ ]:
from IPython import get_ipython
get_ipython().register_magics(MyMagics)

In [ ]:
%%hissp
(define f (lambda (name) {:key name}))

In [ ]:
f("ha")

In [ ]:
%%inter
(html [:h1 "Hello world!"])

In [ ]:
%%inter
(html [:svg {:width 200 :height 100} [:circle {:cx 50 :cy 50 :r 25 :style "fill:red"}]])

In [ ]:
%%hissp
(html [:svg {:width 200 :height 100} [:circle {:cx 50 :cy 50 :r 25 :style "fill:blue"}]])

In [ ]:
def subs(s, m):
    return s[m:]

def plus(a, b):
    return a + b

num = 1

In [ ]:
%%inter
(html [:h3 (subs (subs "hi_hiccup" (plus 1 num)) 1)])

In [ ]:
%%inter
[:h3 (subs (subs "hi_hiccup" (plus 1 num)) 1)]

In [ ]:
%%hissp
(html [:h3 (subs (subs "hi_hissp" (plus 1 num)) 1)])

In [ ]:
%%hissp
[:h3 (subs (subs "hi_hissp" (plus 1 num)) 1)]

In [ ]:
%%hissp0
(html [:h3 (subs (subs "hi_hissp" (plus 1 num)) 1)])

In [ ]:
%%hissp1
(html [:h3 (subs (subs "hi_hissp" (plus 1 num)) 1)])

In [ ]:
%%html
<script src="https://unpkg.com/blockly/blockly_compressed.js"></script>

In [ ]:
%%js
Blockly.defineBlocksWithJsonArray([
     {
         "type": "num",
         "message0": "%1",
         "args0": [
             {
                 "type": "field_input",
                 "name": "nummer",
                 "text": ""
             }
         ],
         "output": null,
         "colour": "#A65C81"
     },
     {
         "type": "funs-h-2-inp",
         "message0": "%1 %2",
         "args0": [
             {
                 "type": "field_input",
                 "name": "kopf",
                 "text": ""
             },
             {
                 "type": "input_value",
                 "name": "args-2"
             }
         ],
         "inputsInline": true,
         "output": null,
         "colour": 270
     },
     {
         "type": "funs-h-3-inp",
         "message0": "%1 %2 %3",
         "args0": [
             {
                 "type": "field_input",
                 "name": "kopf",
                 "text": ""
             },
             {
                 "type": "input_value",
                 "name": "args-2"
             },
             {
                 "type": "input_value",
                 "name": "args-3"
             }
         ],
         "inputsInline": true,
         "output": null,
         "colour": 140
     }
     ]);

In [ ]:
%%html
<xml id="toolbox" style="display: none">
    <category name=">">
        <block type="num"></block>
        <block type="funs-h-2-inp"></block>
        <block type="funs-h-3-inp"></block>
    </category>
</xml>

<div id="blocklyDiv1" style="height: 100%"></div>

<script>
 var workspace1 = Blockly.inject('blocklyDiv1', {
     toolbox: document.getElementById('toolbox') });
</script>
<h1>1</h1><h1>2</h1><h1>3</h1><h1>4</h1><h1>5</h1>

In [ ]:
%%html
<div id="blocklyDiv2" style="height:100%"></div>
<script>
 var workspace2 = Blockly.inject('blocklyDiv2', {
     toolbox: document.getElementById('toolbox') });
</script>
<h1>1</h1><h1>2</h1><h1>3</h1><h1>4</h1><h1>5</h1>